<a href="https://colab.research.google.com/github/RockyHorses/Doom/blob/master/AIProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

#This will definately be useful http://files.pushshift.io/reddit/comments/daily/ 
#That is a huge chat dump from reddit, by huge i mean it contains over 20gb of chat logs, each log is about 300mb
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
import tflearn
import sqlite3
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist
import json
import io
from datetime import datetime
from google.colab import drive

drive.mount('/content/drive/')
!ls "/content/drive/My Drive"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# CHANGE timef IF CHANGING DATASET DATE
#------------------------------------------
timef = '2011-08'
# -----------------------------------------
sql_transfer = []
start_row = 0
cleanup = 1000000 #1 MIL
connection = sqlite3.connect('{}.db'.format(timef))
cursor = connection.cursor()

def create_table():
  cursor.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")
  
def format_data(data):
  date = data.replace("\n"," newlinechar ").replace("\r"," newlinechar ").replace('"',"'")
  return data

def transaction_bldr(sql):
    global sql_transfer
    sql_transfer.append(sql)
    if len(sql_transfer) > 1000:
        cursor.execute('BEGIN TRANSACTION')
        for s in sql_transfer:
            try:
                cursor.execute(s)
            except:
                pass
        connection.commit()
        sql_transfer = []
        
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s-UPDATE insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-PARENT insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-NO_PARENT insertion',str(e))   

def find_existing_score(pid):
  try:
    sql = "Select score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
    cursor.execute(sql)
    result = cursor.fetchone()
    if result != None:
      return result[0]
    else: return False
  except Exception as e:
    #print("find_parent", e)
    return False

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True
  
def find_parent(pid):
  try:
    sql = "Select comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
    cursor.execute(sql)
    result = cursor.fetchone()
    if result != None:
      return result[0]
    else: return False
  except Exception as e:
    #print("find_parent", e)
    return False
        
  
if __name__ == "__main__":
  create_table()
  row_counter = 0
  paired_rows = 0
  
  with open ("/content/drive/My Drive/RC_{}.txt".format(timef.split('9')[0], timef), buffering=1000) as f:  
    for row in f:
     # print(row)
      row_counter += 1 
      row = json.loads(row)
      parent_id = row['parent_id'].split('_')[1]
      body = format_data(row['body'])
      created_utc = row['created_utc']
      score = row ['score']
      subreddit = row['subreddit']
      comment_id = row['id']
      parent_data = find_parent(parent_id)
      
      # UPVOTE SCORE FILTER
      if score >= 5:
        if acceptable(body):
          existing_comment_score = find_existing_score(parent_id)
          if existing_comment_score:
            if score > existing_comment_score:
              ###########
              sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
              
          else:
            if parent_data:
              ###########
              sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
              paired_rows += 1
            else:
              ###########
              sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
              
             
      if row_counter % 100000 == 0:
        print("Total rows read so far: {}, Paired rows: {}, Time: {}".format(row_counter, paired_rows, str(datetime.now())))
            
      if row_counter > start_row:
        if row_counter % cleanup == 0:
          print("Finished 100k rows! Cleaning up...")
          sql = "DELETE FROM parent_reply WHERE parent IS NULL"
          cursor.execute(sql)
          connection.commit()

    100% |████████████████████████████████| 993kB 18.9MB/s 
Instructions for updating:
Colocations handled automatically by placer.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
'Colab Notebooks'   RC_2011-08.txt   RC_2018-02-28.txt	 Untitled0.ipynb
Total rows read so far: 100000, Paired rows: 1012, Time: 2019-03-17 02:16:53.090375
Total rows read so far: 200000, Paired rows: 1990, Time: 2019-03-17 02:16:59.698839
Total rows read so far: 300000, Paired rows: 3138, Time: 2019-03-17 02:17:05.683903
Total r

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))